In [1]:
import os
import pandas as pd
from natural_pdf import PDF

SPORT_CLIP_SETTINGS = {
    "Baseball": lambda page: (
        page.find("text:contains('Baseball')")
        .below().find("text:contains('%-N')")
        .below(until="text:contains('Football')", include_endpoint=False)
        .clip(left=50, right=230)
    ),
    "Football": lambda page: (
        page.find("text:contains('Football')")
        .below(until="text:contains(\"Women's Basketball\")", include_endpoint=False)
        .clip(left=50, right=230)
    ),
    "Men's Basketball": lambda page: (
        page.find("text:contains(\"Men's Basketball\")")
        .below(until="text:contains(\"Men's Other\")", include_endpoint=False)
        .clip(left=230, right=400)
    ),
    "Men's Other": lambda page: (
        page.find("text:contains(\"Men's Other\")")
        .below(until="text:contains(\"Women's CC/Track\")", include_endpoint=False)
        .clip(left=210, right=400)
    ),
    "Men's CC/Track": lambda page: (
        page.find("text:contains(\"Men's CC/Track\")")
        .below(until="text:contains(\"Women's Other\")", include_endpoint=False)
        .clip(left=405, right=575)
    ),
    "Women's Basketball": lambda page: (
        page.find("text:contains(\"Women's Basketball\")")
        .below(until="text:contains(\"Values\")", include_endpoint=False)
        .clip(left=50, right=230)
    ),
    "Women's CC/Track": lambda page: (
        page.find("text:contains(\"Women's CC/Track\")")
        .below(until="text:contains(\"Values\")", include_endpoint=False)
        .clip(left=225, right=400)
    ),
    "Women's Other": lambda page: (
        page.find("text:contains(\"Women's Other\")")
        .below(until="text:contains(\"Values\")", include_endpoint=False)
        .clip(left=405, right=575)
    ),
}

def extract_school_name(filename):
    return filename.replace(".pdf", "").replace("_", " ").strip()

def clean_row(row):
    return [cell.strip().replace("\xa0", " ") if isinstance(cell, str) else "" for cell in row]

def extract_all_sports(pdf, school_name):
    page = pdf.pages[1]
    all_rows = []
    for sport, extractor in SPORT_CLIP_SETTINGS.items():
        try:
            table = extractor(page).extract_table()
            if not table or len(table) < 2:
                continue
            for row in table:
                row = clean_row(row)
                row_padded = row[:5] + [""] * (5 - len(row))  # pad to 5 columns
                all_rows.append({
                    "School": school_name,
                    "Sport": sport,
                    "Race": row_padded[0],
                    "%N": row_padded[1],
                    "2017-18": row_padded[2],
                    "4-Class": row_padded[3],
                    "GSR": row_padded[4],
                })
        except Exception as e:
            print(f"{sport} failed for {school_name}: {e}")
    return all_rows

pdf_folder = "pdfs"
pdf_files = sorted([f for f in os.listdir(pdf_folder) if f.endswith(".pdf")])
all_data = []

for filename in pdf_files:
    school_name = extract_school_name(filename)
    filepath = os.path.join(pdf_folder, filename)
    try:
        pdf = PDF(filepath)
        if len(pdf.pages) < 2:
            print(f"Skipping {school_name} (too few pages)")
            continue
        data = extract_all_sports(pdf, school_name)
        all_data.extend(data)
        print(f"Finished {school_name}")
    except Exception as e:
        print(f"Failed {school_name}: {e}")

df = pd.DataFrame(all_data)
df.to_csv("Page2_AllSchools.csv", index=False)
print("Saved to Page2_AllSchools.csv")

KeyboardInterrupt: 